In [294]:
import cv2 as cv
import numpy as np
import random
import time
from google.colab.patches import cv2_imshow

In [295]:
alpha = 0.5
random_seed = 2022

In [296]:
blank_image = np.zeros((20,20,3), np.uint8)
blank_image[:] = (255,0,0)
cv.imwrite("new.png",blank_image)

True

In [297]:
source_image_path = "new.png"
source_image = cv.imread(source_image_path)
cv2_imshow(source_image)

In [298]:
text_to_hide = "Te iubesc, Ana"
text_array = list(text_to_hide)
text_array = list(map(lambda x: ord(x),text_array))
text_array = np.array(text_array)
text_array.astype(dtype='uint8')



array([ 84, 101,  32, 105, 117,  98, 101, 115,  99,  44,  32,  65, 110,
        97], dtype=uint8)

In [305]:
def encode(img_path,text):
  img = cv.imread(img_path)
  img_height,img_width,_ = img.shape

  text_array = list(text)
  text_array = list(map(lambda x: ord(x),text_array))
  text_array = np.array(text_array)
  text_array.astype(dtype='uint8')

  hide_text_dim = len(text)
  
  # Get random seed
  y_random_indices, x_random_indices = list(range(img_height)), list(range(img_width))
  random.seed(random_seed)
  random.shuffle(x_random_indices)
  random.shuffle(y_random_indices)

  # Injection watermark
  hide_layer = np.zeros(img.shape, dtype=np.uint8)
  for x in range(hide_text_dim):
    hide_layer[y_random_indices[x], x_random_indices[x]] = text_array[x]

  img_freq = np.fft.fft(img)
  encoded_freq = img_freq + hide_layer

  new_img = np.fft.ifft(encoded_freq)
  new_img = np.real(new_img).astype(np.uint8)

  global copy_img
  copy_img = new_img

  cv2_imshow(new_img)
  cv.imwrite("encoded.png",new_img)



In [306]:
 encode(source_image_path,text_to_hide)

In [303]:
def decode(src_path,src_encoded):
  img = cv.imread(src_path)
  img_height,img_width,_ = img.shape

  img_encoded = cv.imread(src_encoded)
  
  # Get random seed
  y_random_indices, x_random_indices = list(range(img_height)), list(range(img_width))
  random.seed(random_seed)
  random.shuffle(x_random_indices)
  random.shuffle(y_random_indices)

  img_freq = np.fft.fft(img)
  encoded_freq = np.fft.fft(img_encoded)

  hidden_layer = encoded_freq - img_freq

  hidden_layer = np.real(hidden_layer).astype(np.uint8)

  dim_text = 0
  for rows in hidden_layer:
    for elem in rows:
      if elem[0] != 0:
        dim_text += 1

  recreated_text = []
  for i in range(dim_text):
    recreated_text.append("0")
  for i in range(dim_text):
    recreated_text[i] = hidden_layer[y_random_indices[i],x_random_indices[i]][0]

  decoded_text = ''.join(map(lambda x : chr(x),recreated_text))
  print(decoded_text)


In [304]:
encoded_image = "encoded.png"
decode(source_image_path,encoded_image)

Te iubesc, Ana
